In [1]:
# ------------------------------------
# 
# testing a setup of method of morris
# sampling for the lake ebm
# 
# ------------------------------------
import os 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from SALib.sample import morris as morris_sampler
from SALib.analyze import morris as morris_analyzer

## input variable notes
See [here](https://docs.google.com/presentation/d/1ars5brZvXNqyo5ME6HpbfY23IXhvAeWvC888YLU7dEQ/edit?slide=id.g358672e6939_0_12#slide=id.g358672e6939_0_12) for more details. 

#### Vars we tested
- `basedep` (basin depth): ignore, no effect on T

- `b_area` (basin area): ignore, no effect on T

- `max_dep` (max spillover depth): ignore, negligible effect on T

- `tempinit` (initial lake temperature): ignore, it matters but only numerically (not physically)

- `depth_begin` (lake depth): **include**, large effect on column T (smaller effect on LST)

- `salty_begin` (salinity): **include**, small effect on T (can be about 1 deg)


#### Climate vars (anoms relative to CTRL)
- `TSplus` (temperature plus this value): obviously matters, probably should include it!  

- `RHplus` (relhum plus this value): need to make sure we impose bounds at 1 and 99

- `Ufac` (wind speed times this factor): large effect on temperature depending on the scale you give it

- `FSDSplus` (FSDS plus this value)

- `FLDSplus` (FLDS plus this value)

- `PSplus` (PS plus this value): very tiny effect on temperature, but physically relevant so we'll keep it

- `PRECTplus` (PRECT plus this value): No effect on temperature according to morris SA

#### Other vars
- `oblq` (obliquity): No effect on temperature according to morris SA

- `xlat` (latitude): relatively small effect on temperature... not sure what it is..? omit for now?

- `xlon` (longitude): No effect on temperature according to morris SA

- `cdrn` (neutral drag coeff): untested; assume range 1e-3 to 2e-3 (1.8 HAD 1.7GISS 1.2CCSM)

- `eta` (shortwave extinction coeff): untested; see [here](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2014JD022938)

- `f` (fract air advected over lake): No effect on temperature according to morris SA

- `gmt` (timezone): ignore, technically constrained by lat / lon

- `alb_slush` (albedo of slush): untested, ignore because we have variable ice cover turned off

- `alb_snow` (albedo of snow): untested, ignore because we have variable ice cover turned off


In [2]:
# --- Set up the problem; define model inputs
# 
# note on climate vars:
# TSplus: temperature plus this value

problem = {
    "num_vars": 10,
    "names": ["depth_begin", "salty_begin", "cdrn",       "eta",       "TSplus", "RHplus",   
                "Ufac",   "FSDSplus", "FLDSplus",   "PSplus", ],
    "bounds": [[10, 200], [0.5, 110],       [1e-3, 2e-3], [0.03, 3.0], [-6, 6], [-15, 15], 
               [0.7, 1.3], [-25, 25],  [-50, 50],  [-1000, 14000]]  
}


In [3]:
# --- generate samples using morris sampler
param_values = morris_sampler.sample(
    problem, 
    N=1000, 
    num_levels=4, 
    optimal_trajectories=65,
    seed = 1111,
)
len(param_values)

715

# Build the batch file 

In [14]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++
# --- NAME THE SENSITIVITY EXPERIMENT
iters = str(len(param_values))
tag = None
thisversion = "v1"
if tag is None:
    exp_name = f"SAmorris{iters}_{thisversion}"
else:
    exp_name = f"SAmorris{iters}_{tag}_{thisversion}"
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++


# --- where to save the batch file
SAVE_ON = True
savehere = "/Users/tylerkukla/Documents/GitHub/PRYSM/psm/lake_v2/batch_inputs"
savename = f"batch_CP_sensitivity_morris-{iters}iter_{thisversion}.csv"

# --- where to save the SA settings
SAsetting_path = "/Users/tylerkukla/Documents/GitHub/PRYSM/psm/lake_v2/output/batch/_SA-settings"

In [15]:
# --- save the SA settings --------------------------------------
if SAVE_ON:
    savesettings_here = os.path.join(SAsetting_path, exp_name)
    if not os.path.exists(savesettings_here):
        os.makedirs(savesettings_here)  
        
    # save array
    np.save(os.path.join(savesettings_here, 'param_values.npy'), param_values)
    # save dict
    pd.to_pickle(problem, os.path.join(savesettings_here, "problem.pkl"))
    # with open(os.path.join(savesettings_here, "problem.pkl"), "wb") as f:
    #     pickle.dump(problem, f)
# ---------------------------------------------------------------

In [16]:
# --- 
df = pd.DataFrame(param_values, columns = problem['names'])
# make depth_begin an integer since depths are in 1m intervals
df['depth_begin'] = df['depth_begin'].apply(lambda x: int(round(x)) if isinstance(x, float) else x)

# add default row for the control case
df.loc[len(df)] = ["**default**"] * len(df.columns)
df

,depth_begin,salty_begin,cdrn,eta,TSplus,RHplus,Ufac,FSDSplus,FLDSplus,PSplus
0,10,110.0,0.002,2.01,2.0,5.0,1.1,8.333333,16.666667,4000.0
1,10,110.0,0.002,0.03,2.0,5.0,1.1,8.333333,16.666667,4000.0
2,10,110.0,0.002,0.03,2.0,5.0,1.1,-25.0,16.666667,4000.0
3,10,110.0,0.002,0.03,2.0,-15.0,1.1,-25.0,16.666667,4000.0
4,10,37.0,0.002,0.03,2.0,-15.0,1.1,-25.0,16.666667,4000.0
...,...,...,...,...,...,...,...,...,...,...
711,10,73.5,0.001,2.01,-6.0,-15.0,0.9,25.0,-16.666667,-1000.0
712,10,73.5,0.001,0.03,-6.0,-15.0,0.9,25.0,-16.666667,-1000.0
713,10,73.5,0.001,0.03,-6.0,-15.0,0.9,-8.333333,-16.666667,-1000.0
714,10,73.5,0.001,0.03,-6.0,-15.0,0.9,-8.333333,-16.666667,9000.0


In [17]:
# --- function to add constant values to the dict
def add_constant_parameters(
        df_batch: pd.DataFrame,
        constant_dict: dict,
)->pd.DataFrame:
    '''
    Take in the existing batch dataframe and add the constant parameter 
    values. 

    Parameters
    ----------
    df_batch : pd.DataFrame
        the pandas dataframe that is output from one of the other sample 
        functions (latin_hypercube_sampler, all_combinations_sampler,
        prescribed_cases).
    constant_dict : dict
        dictionary where keys are parameter names and each has a single 
        value that is held constant for all rows. 

    Returns
    -------
    pd.DataFrame
        the final batch .csv that gets saved
    '''
    # check that all dicts have only one value
    all_len_1 = all((isinstance(value, (list, tuple)) and len(value) == 1) or not isinstance(value, (list, tuple)) for value in constant_dict.values())
    if not all_len_1:
        warnings.warn("Expected all dict parameters to have one value but at least one parameter has more. This may lead to unintended results.", UserWarning)

    # constant DataFrames
    df2 = pd.DataFrame([constant_dict])

    # Create a dummy key for cross join
    df_batch['key'] = 1
    df2['key'] = 1

    # Merge the DataFrames on the dummy key
    combined_df = pd.merge(df_batch, df2, on='key').drop('key', axis=1)
    return combined_df


    '''
    Add experiment names to the one_at_a_time output with the pattern
    <parameter_name><counter>. For example, if the parameter that is not
    default is "max_dep" and it's the first one in the df, the name will 
    be max_dep1. 

    Note, the code identifies which column's value is NOT == the default
    string. If there is more than one, it takes the first column name. 
    '''
    exp_names = []
    counters = {col: 0 for col in df.columns if col != "exp_name"}

    for _, row in df.iterrows():
        for col in counters:
            if row[col] != default_str:
                counters[col] += 1
                exp_names.append(f"{col}{counters[col]}")
                break

    df = df.copy()
    df["casename"] = exp_names
    return df

In [18]:
# --- add constant values
constant_dict = {
    # --- THE ONLY TWO REQUIRED COLUMNS ---
    # [UPDATE TO PATH ON YOUR MACHINE]
    "default_dict_path": "/Users/tylerkukla/Documents/GitHub/PRYSM/psm/lake_v2/defaults",
    "dict_name": "defaults_CP",
    # -------------------------------------
    # 
    # other constants
    "datafile": "CP_SLIM_modernTopo_280ppm_input.txt",
    "outdir": "/Users/tylerkukla/Documents/GitHub/PRYSM/psm/lake_v2",
    # 
    # make sure we update climatology! 
    "update_clim": True,
}

# --- add to the dataframe
dfout = add_constant_parameters(df, constant_dict)
dfout

,depth_begin,salty_begin,cdrn,eta,TSplus,RHplus,Ufac,FSDSplus,FLDSplus,PSplus,default_dict_path,dict_name,datafile,outdir,update_clim
0,10,110.0,0.002,2.01,2.0,5.0,1.1,8.333333,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
1,10,110.0,0.002,0.03,2.0,5.0,1.1,8.333333,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
2,10,110.0,0.002,0.03,2.0,5.0,1.1,-25.0,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
3,10,110.0,0.002,0.03,2.0,-15.0,1.1,-25.0,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
4,10,37.0,0.002,0.03,2.0,-15.0,1.1,-25.0,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,10,73.5,0.001,2.01,-6.0,-15.0,0.9,25.0,-16.666667,-1000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
712,10,73.5,0.001,0.03,-6.0,-15.0,0.9,25.0,-16.666667,-1000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
713,10,73.5,0.001,0.03,-6.0,-15.0,0.9,-8.333333,-16.666667,-1000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
714,10,73.5,0.001,0.03,-6.0,-15.0,0.9,-8.333333,-16.666667,9000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True


In [19]:
# --- add unique name for each experiment
dfout['casename'] = dfout.apply(lambda row: f"{exp_name}_{row.name + 1}", axis=1)
dfout = dfout[['casename'] + [col for col in dfout.columns if col != 'casename']]
dfout

,casename,depth_begin,salty_begin,cdrn,eta,TSplus,RHplus,Ufac,FSDSplus,FLDSplus,PSplus,default_dict_path,dict_name,datafile,outdir,update_clim
0,SAmorris715_v1_1,10,110.0,0.002,2.01,2.0,5.0,1.1,8.333333,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
1,SAmorris715_v1_2,10,110.0,0.002,0.03,2.0,5.0,1.1,8.333333,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
2,SAmorris715_v1_3,10,110.0,0.002,0.03,2.0,5.0,1.1,-25.0,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
3,SAmorris715_v1_4,10,110.0,0.002,0.03,2.0,-15.0,1.1,-25.0,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
4,SAmorris715_v1_5,10,37.0,0.002,0.03,2.0,-15.0,1.1,-25.0,16.666667,4000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,SAmorris715_v1_712,10,73.5,0.001,2.01,-6.0,-15.0,0.9,25.0,-16.666667,-1000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
712,SAmorris715_v1_713,10,73.5,0.001,0.03,-6.0,-15.0,0.9,25.0,-16.666667,-1000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
713,SAmorris715_v1_714,10,73.5,0.001,0.03,-6.0,-15.0,0.9,-8.333333,-16.666667,-1000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True
714,SAmorris715_v1_715,10,73.5,0.001,0.03,-6.0,-15.0,0.9,-8.333333,-16.666667,9000.0,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,defaults_CP,CP_SLIM_modernTopo_280ppm_input.txt,/Users/tylerkukla/Documents/GitHub/PRYSM/psm/l...,True


In [20]:
# --- save the result
if SAVE_ON:
    savefile = os.path.join(savehere, savename)
    dfout.to_csv(savefile, index=False)
    print(f"Saved batch file to {savefile}")

Saved batch file to /Users/tylerkukla/Documents/GitHub/PRYSM/psm/lake_v2/batch_inputs/batch_CP_sensitivity_morris-715iter_v1.csv


In [ ]:
# ----------------------